<a href="https://colab.research.google.com/github/pachterlab/BI-BE-CS-183-2023/blob/main/HW6/Problem3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bi/Be/Cs 183 2022-2023: Intro to Computational Biology
TAs: Meichen Fang, Tara Chari, Zitong (Jerry) Wang

**Submit your notebooks by sharing a clickable link with Viewer access. Link must be accessible from submitted assignment document.**

Make sure Runtime $\rightarrow$ Restart and run all works without error

**HW 6 Problem 3**

In this problem you will test different methods for variance stabilization on real single-cell datasets, and analyze the results of these procedures. This follows a recent [paper](https://www.biorxiv.org/content/biorxiv/early/2021/08/25/2021.06.24.449781.full.pdf) and [blog post](https://www.nxn.se/valent/2017/10/15/variance-stabilizing-scrna-seq-counts) about their effects in single-cell.


##**Import data and install packages**

In [ ]:
import numpy as np
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt #Can use other plotting packages like seaborn

In [ ]:
#Download the gene count matrix for Drop-seq Drospohila embryo data
!wget --content-disposition https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM2494nnn/GSM2494783/suppl/GSM2494783_dge_mel_vir_rep1.txt.gz

--2022-02-08 17:37:53--  https://ftp.ncbi.nlm.nih.gov/geo/samples/GSM2494nnn/GSM2494783/suppl/GSM2494783_dge_mel_vir_rep1.txt.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.230, 130.14.250.7, 2607:f220:41f:250::230, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6388719 (6.1M) [application/x-gzip]
Saving to: ‘GSM2494783_dge_mel_vir_rep1.txt.gz’

GSM2494783_dge_mel_ 100%[===================>]   6.09M  33.8MB/s    in 0.2s    

2022-02-08 17:37:54 (33.8 MB/s) - ‘GSM2494783_dge_mel_vir_rep1.txt.gz’ saved [6388719/6388719]



In [ ]:
!gunzip GSM2494783_dge_mel_vir_rep1.txt.gz

## **Read in data for analysis**

**The dataset**

This dataset is from a Drop-seq experiment whose purpose was to conduct a single-cell study of the early *Drosophila* (fruit fly) embryo at particular stages of development ([Karaiskos et al., 2017](http://dx.doi.org/10.1126/science.aan3235)), from both *Drosophila melanogaster* and *Drosophila virilis* species. Over 5000 embryos were sequenced to  generate a predictive 3D map of gene expression during development across the embryo (using previous *in situ* hybridization data).

<center><img src="https://drive.google.com/uc?export=view&id=1p4qrvbhjGahIQL1s3M-UzAFbqhuNyTt7" alt="EMFigure" width="600" height="300"><center>



**The count matrix**

The gene count matrix is 3,247 cells by 23,712 genes. These counts have not been processed/normalized, so they directly represent the UMI counts from each cell.


In [ ]:
#Get gene count matrix
data = pd.read_csv('GSM2494783_dge_mel_vir_rep1.txt', sep='\t',index_col=0)
data.head()

,CATCTTGGTTCN,GTACTAATTACN,GGAAACACGTTC,ACGCACAACTCN,AGAGCTCGTGTA,AATCACCTCCAA,CATAATTTAGCT,GTGTATTTGTCN,TTCTTCACTTTC,CCAGTGTCTTGC,CAGAAGTTGCCG,TACTAGGTTTTC,GCCTTTTGGTTG,AGGCTAATGGAC,CCTGAATTTATN,CTTGCTATAACC,CTCGAGTCCTAA,GGATATCGACCA,GAAGCATTAACT,GAAGAGAACGTT,ATGGTCTATCAC,CTACTGTCAGGA,CCTGAAATTTAT,AACTAAACCATA,TCTAGTTACGCG,TAGACAAAAGCT,CTTAAGCGCGTC,TCTACTAGTGTN,CGTTGTTATGCT,ATTGACAGGGTC,CCGTGCTGAACA,TCCTCCATTCCA,GAGAAAATGAAG,TTGCGACCCAAN,AGCCCCCCGGGA,GCCTAGTGACGT,TTGTTAAAGTCG,CAGTATCGAGAN,CAAAGTATTCGG,CAGCCATCTCCC,...,TGATGATCTGTT,CAAACCTATACT,CCTATACTGGCC,TTACATATATGN,CCTTTTTACGTT,GTTAATGCATTC,TTATAATTGCAA,CTAGTGTAGTCT,TGGCACGATCAT,CAACAAGAGTGT,ACCTCGCGTGGN,CGACAGAGAGGA,CGGCCAGCGCAT,GTGTAGAACCTG,GAGTCACATCGC,AATGATCCGTGC,TCCAAGACCTGG,GTTGCATTTGGC,GTCGATCTACGC,CCGCGTGGTAAT,ATTAAGATATTG,CGGTGAACTAAT,TTTAGCAGTCTT,TACTCAAGAGAC,CAGTGAATAACT,GTATGTGTTTCA,ACTAGCAATGTA,CGGTGCGATAAA,CTGTATGCTCGG,GTAACGAATTAN,TTCCCTAGGTAA,CCTGTAGCGATA,TAAGGGCGCCTC,ATCTGACCAGAA,ATTCCCACTCGT,ATTCCTTATTAG,CGGTAAGCAGGC,AGCAATGAGTCT,CTTCACCTAAGA,TCGCTAATGCCN
GENE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
128up,6,4,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
14-3-3epsilon,665,370,4,5,1,1,4,1,0,8,4,1,209,188,2,1,2,0,5,1,1,1,2,1,0,115,1,119,2,0,92,2,120,1,6,107,51,95,2,2,...,0,0,4,0,5,3,1,0,3,0,0,1,0,0,1,6,0,3,3,0,6,1,2,4,1,1,8,4,0,0,4,2,0,0,1,0,0,1,3,1
14-3-3zeta,120,49,0,1,0,0,1,0,0,4,0,0,39,18,0,3,0,0,0,0,0,0,0,0,0,16,0,8,0,1,18,0,13,0,1,12,7,8,0,0,...,0,0,0,0,0,2,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
140up,6,1,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,2,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18SrRNA-Psi:CR41602,4,0,3,0,1,0,1,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Extract just the counts and transpose (to get cells x genes)
count_mat = data.to_numpy().T
count_mat.shape

(3247, 23712)

## **Problem 3 (40 points)**

For our purposes, we will use the $\mu,\phi$ parametrization of the negative binomial (NB) for this problem. Here $\phi$ is the dispersion and $\mu$ is the mean.

In this configuration, $\operatorname {var}(X) = \mu + \phi\mu^2$ (unlike the Poisson where $\operatorname{var}(X) = \mu$). $x_i$ represents expression of gene $i$.


As described in the assignment, we can find a variance-stabilizing transform, where given
\begin{align}
\operatorname {var} (X)=h(\mu ),\,
\end{align}
a suitable transform would be
\begin{align}
 y\propto \int ^{x}{\frac {1}{\sqrt {h(\mu )}}}\,d\mu 
\end{align}
to result in a constant (mean-independent) variance.


### **a) Find the expression for the transformation $y$ given the var$(X)$ expression for a NB (given in the Problem statement). (5 points)**

If working by hand attach an image of your work, or directly type your answer into a text cell. Feel free to use https://www.wolframalpha.com/ for the integral calculation.

### **b) Run PCA on the data matrix (with genes as features), extract the top two principal components and transform the data matrix, then plot the cells in their 2D, transformed coordinates. (5 points)**

You can use the [sklearn PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) function, similar to HW3.

### **c) Plot the variance ($\sigma^2$) versus the mean ($\mu$) expression for all genes in a single plot, and comment on any trends you notice (how variance relates to the mean). (5 points)**

You will need to calculate a $\mu$ and $\sigma^2$ for each gene.

### **d) Fit a polynomial to $\sigma^2$ vs $\mu$ (the plot from c) to approximate a single $\phi$ value (across all genes). (5 points)**

$\sigma^2$ is var$(X)$. Given that  $\operatorname {var}(X) = \mu + \phi\mu^2$ you can find the fit for $\phi$ as the coefficent for the squared term.

You can use the package [curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) to define this degree 2 polynomial where $x=\mu$, $y=$ var$(X)$. Using the bounds options you can constrain the constant to be 0, the first coefficient to be 1, and learn the second coefficient ($\phi$). 

See constrained fit example [here](https://stackoverflow.com/questions/48469889/how-to-fit-a-polynomial-with-some-of-the-coefficients-constrained).

**Report your value of $\phi$ from the fit (one value across all genes).**

In [ ]:
from scipy.optimize import curve_fit

### **e) Run the log1p, Pearson residual, and $\mathbf{\text{sinh}^{-1}}$ variance stabilization transforms on the full dataset. (10 points)**

Below you will test out the effect of common variance-stabilization procedures.

[In 1948](https://academic.oup.com/biomet/article-abstract/35/3-4/246/280278?redirectedFrom=fulltext), Frank Anscombe developed several transformations for the Poisson and NB distributions including

\begin{align}
y \propto \dfrac{\text{sinh}^{-1}(\sqrt{\phi x_i})}{\sqrt{\phi}} \tag{1}
\end{align} and
\begin{align}
y \propto \text{log}(x_i+\dfrac{1}{2\phi}) \tag{2}
\end{align} (similar to the log1p we've seen before) which can approximate the $\text{sinh}^{-1}$ solution.



Another common method is to use Pearson residuals, shown below:

\begin{align}
y \propto \dfrac{x_i − \mu_i}{\sqrt{\mu_i + \phi \mu_i^2}}. \tag{3}
\end{align}

Again $x_i$ represents expression of gene $i$.

**After running each transformation (on the full data), print *only* the transformed values for the first gene, for the first 10 cells, under each transform (1-3).**

### **f) For each of the three transformation methods, make a single plot of the variance ($\sigma^2$) versus the mean ($\mu$) for all genes, and comment on the trends you notice (particularly compared to c). (5 points)**

### **g) For each transformation, run PCA on the variance-stabilized data matrices (with genes as features), extract the top two principal components and transform the matrix, then plot the cells in their 2D, transformed coordinates. There should be one plot for each transformation method. Comment on how these plots compare to that of b. (5 points)**